### To robustly handle non-matching images image matrices and spacing

-  Resample moving images (pre/AP) to match the fixed image (PP)


In [1]:
import os
import SimpleITK as sitk
from tqdm import tqdm

In [2]:
# Path configuration
base_path = "E://data//single_TARE_mismatch"
output_base_path = "E://data//single_TARE_mismatch_resampled_PPfixed"
os.makedirs(output_base_path, exist_ok=True)

In [3]:

# This resampling works only if the FOVs of image and target overlap in physical space
def resample_image_to_target(image, target, interpolator=sitk.sitkLinear):
    resample = sitk.ResampleImageFilter()
    resample.SetReferenceImage(target)  # Automatically sets spacing, size, origin, direction
    resample.SetTransform(sitk.Transform())  # Identity transform
    resample.SetInterpolator(interpolator)
    resample.SetDefaultPixelValue(0)
    return resample.Execute(image)



In [4]:
# Process each patient
for patient_id in tqdm(os.listdir(base_path)):
    patient_dir = os.path.join(base_path, patient_id)
    if not os.path.isdir(patient_dir):
        continue

    try:
        pre_path = os.path.join(patient_dir, "pre.nrrd")
        ap_path = os.path.join(patient_dir, "AP.nrrd")
        pp_path = os.path.join(patient_dir, "PP.nrrd")

        # Load images
        pre = sitk.ReadImage(pre_path)
        ap = sitk.ReadImage(ap_path)
        pp = sitk.ReadImage(pp_path)

        # Resample moving images to match the fixed image
        pre_resampled = resample_image_to_target(pre, pp, interpolator=sitk.sitkLinear)
        ap_resampled = resample_image_to_target(ap, pp, interpolator=sitk.sitkLinear)

        # Output patient directory
        output_patient_dir = os.path.join(output_base_path, patient_id)
        os.makedirs(output_patient_dir, exist_ok=True)

        # Save resampled images
        sitk.WriteImage(pre_resampled, os.path.join(output_patient_dir, "pre.nrrd"))
        sitk.WriteImage(ap_resampled, os.path.join(output_patient_dir, "AP.nrrd"))
        sitk.WriteImage(pp, os.path.join(output_patient_dir, "PP.nrrd"))

    except Exception as e:
        print(f"[Error] {patient_id}: {e}")

100%|██████████| 8/8 [01:18<00:00,  9.83s/it]


### PID 1570909 : origin z axis 너무 차이 나서 forced matching (size/spacing/direction은 일치하였음)

In [5]:
import SimpleITK as sitk
import os

def force_match_geometry(moving_img, reference_img):
    aligned_img = sitk.Image(moving_img)  # Copy image content
    aligned_img.CopyInformation(reference_img)  # Copy spacing, origin, direction
    return aligned_img


patient_dir = "E://data//single_TARE_mismatch//1570909"

# Example:
pre_path = os.path.join(patient_dir, "pre.nrrd")
ap_path = os.path.join(patient_dir, "AP.nrrd")
pp_path = os.path.join(patient_dir, "PP.nrrd")

pre = sitk.ReadImage(pre_path)
ap = sitk.ReadImage(ap_path)
pp = sitk.ReadImage(pp_path)

pre_fixed = force_match_geometry(pre, pp)
ap_fixed = force_match_geometry(ap, pp)

forced_output_dir = "E://data//1570909_forcedz_PPfixed"
os.makedirs(forced_output_dir, exist_ok=True)

sitk.WriteImage(pre_fixed, os.path.join(forced_output_dir, "pre_fixed.nrrd"))
sitk.WriteImage(ap_fixed, os.path.join(forced_output_dir, "AP_fixed.nrrd"))
sitk.WriteImage(pp, "E://data//1570909_forcedz//PP_fixed.nrrd")